In [79]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import torch
#from "Tools.py" import 
seed=42

### load dataset

In [72]:
banco = pd.read_csv("Life_Expectancy_Data.csv")
#x = banco[["infant deaths","Year"]]
x = banco["infant deaths"]
y =banco["Life expectancy "]

### create folders

In [74]:
cv = KFold(n_splits=10,shuffle =True,random_state=seed)
cross = cv.split(x)
x_train = []
x_test = []
y_train = []
y_test = []
for train_i,test_i in cross:
    x_train.append(x[train_i])
    x_test.append(x[test_i])
    y_train.append(y[train_i])
    y_test.append(y[test_i])

### define model

In [101]:
x = x_train[0]
y = y_train[0]

ann = ANN("mod")
ann.fit(x.to_numpy().reshape(-1, 1),y)

array([-0.4537628 , -0.99003918, -0.99003918, ..., -2.57783786,
       -2.52526174, -2.45165518])

In [77]:
?torch.as_tensor

Docstring:
as_tensor(data, dtype=None, device=None) -> Tensor

Converts data into a tensor, sharing data and preserving autograd
history if possible.

If data is already a tensor with the requeseted dtype and device
then data itself is returned, but if data is a
tensor with a different dtype or device then it's copied as if using
`data.to(dtype=dtype, device=device)`.

If data is a NumPy array (an ndarray) with the same dtype and device then a
tensor is constructed using :func:`torch.from_numpy`.

.. seealso::

    :func:`torch.tensor` never shares its data and creates a new "leaf tensor" (see :doc:`/notes/autograd`).


Args:
    data (array_like): Initial data for the tensor. Can be a list, tuple,
        NumPy ``ndarray``, scalar, and other types.
    dtype (:class:`torch.dtype`, optional): the desired data type of returned tensor.
        Default: if ``None``, infers data type from :attr:`data`.
    device (:class:`torch.device`, optional): the device of the constructed tensor. If N

In [99]:
np.array(x).reshape(-1, 1).reshape(len(x))

array([62, 64, 66, ..., 25, 25, 24], dtype=int64)

In [100]:
class ANN:
    def __init__(self,model,random_state=0,intercv = 3,n_jobs = 15,batch_size=16,lr=0.001):
        self.random_state = random_state
        self.intercv = intercv
        self.n_jobs = n_jobs
        self.model = model
        self.batch_size = batch_size
        self.lr = lr
        self.scale_x = StandardScaler()
        self.scale_y = StandardScaler()

    def fit(self, x, y):
        x_ = self.scale_x.fit_transform(np.array(x))
        y_ = self.scale_y.fit_transform(np.array(y).reshape(-1, 1)).reshape(len(y))
        return y_

    def predict(self, x):
        return self.model.predict(x)
    
    def predict_proba(self,x):
        return self.model.predict_proba(x)
    